In [1]:
import logging
import time
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from models.generation import Model

from config import HUGGINGFACE_TOKEN, GPT4ALL_PATH, LLAMA_7B_PATH, LLAMA_13B_PATH

from helpers.data_helpers import save_to_parquet

logger = logging.getLogger(__name__)

2023-04-04 19:03:09,218 - INFO     | config     | Loading environment variables


In [2]:
# cooldown in seconds every 50 iterations and fast cooldown every iteration
COOLDOWN = 120
FAST_COOLDOWN = 2

# Data

In [3]:
tweets = pd.read_parquet('data/eval_tweets_202342.parquet')
logger.info(f'{len(tweets)} tweets in generation set.')

2023-04-04 19:03:10,167 - INFO     | __main__   | 21711 tweets in generation set.


In [4]:
N_TWEETS = 100
logger.info(f'Generating from {N_TWEETS} tweets.')

2023-04-04 19:03:10,352 - INFO     | __main__   | Generating from 100 tweets.


In [5]:
output_tweets = tweets.iloc[:N_TWEETS].copy()
output_tweets.drop(columns=['created_at', 'entities'], inplace=True)

# Prompt Engineering

In [6]:
PROMPT_TEMPLATE = """Answer the question based on the context below. \
    Context: You are a marketing and customer relationship management assistant, \
    your task is to classify a given tweet as either a \
    potential lead or not. Provide your detailed analysis of the following tweet \
    as a potential lead in the context of marketing and customer relationship management. \
    Tweet: {tweet} \
    Question: Is the above tweet a potential lead? Yes or No? Why?. \
    Answer: """

# Models

todo

### BLOOM

In [7]:
bloom = Model(
    model_name='bloom',
    hf_api=HUGGINGFACE_TOKEN
)

2023-04-04 19:03:12,567 - INFO     | models.generation | 
Initializing BLOOM model  - Temp: 1e-10 - Context window: 2048 - Max tokens: 256


In [8]:
bloom.init_prompt(template=PROMPT_TEMPLATE, input_vars=['tweet'])

2023-04-04 19:03:13,511 - INFO     | models.generation | Injecting Variables: ['tweet']


'Answer the question based on the context below.     Context: You are a marketing and customer relationship management assistant,     your task is to classify a given tweet as either a     potential lead or not. Provide your detailed analysis of the following tweet     as a potential lead in the context of marketing and customer relationship management.     Tweet: {tweet}     Question: Is the above tweet a potential lead? Yes or No? Why?.     Answer: '

In [12]:
logger.info('Starting BLOOM generation...')
bloom_outs = [' ' for _ in range(N_TWEETS)]
for i, tweet in enumerate(tqdm(output_tweets['full_text'])):
    bloom_llm, bloom_out = bloom.generate(inject_obj=tweet)
    bloom_outs[i] = bloom_out
    time.sleep(FAST_COOLDOWN)
    if (i+1) % 50 == 0:
        logger.info(f'{i+1} - Saving checkpoint...')
        output_tweets['bloom_out'] = np.array(bloom_outs)
        save_to_parquet(data_dir='outputs', df=output_tweets, name='output_tweets')

2023-04-04 19:14:27,977 - INFO     | __main__   | Starting BLOOM generation...


  0%|          | 0/100 [00:00<?, ?it/s]

2023-04-04 19:16:33,421 - INFO     | __main__   | 50 - Saving checkpoint...


ArrowTypeError: ("Expected bytes, got a 'int' object", 'Conversion failed for column bloom_out with type object')

In [ ]:
output_tweets['bloom_out'] = bloom_outs
save_to_parquet(data_dir='outputs', df=output_tweets, name='output_tweets')

### GPT4All

In [10]:
gpt4all = Model(
    model_name='gpt4all',
    n_threads=6,
    local_model_path=GPT4ALL_PATH
)

2023-04-04 17:25:10,396 - INFO     | models.generation | 
Initializing GPT4ALL model  - Temp: 1e-10 - Context window: 2048 - Max tokens: 256
llama_model_load: loading model from '/Users/lorenzo/Documents/repos/msc-thesis-llm-clustering/.models/gpt4all-7B/gpt4all-converted.bin' - please wait ...
llama_model_load: n_vocab = 32001
llama_model_load: n_ctx   = 2048
llama_model_load: n_embd  = 4096
llama_model_load: n_mult  = 256
llama_model_load: n_head  = 32
llama_model_load: n_layer = 32
llama_model_load: n_rot   = 128
llama_model_load: f16     = 2
llama_model_load: n_ff    = 11008
llama_model_load: n_parts = 1
llama_model_load: type    = 1
llama_model_load: ggml map size = 4017.70 MB
llama_model_load: ggml ctx size =  81.25 KB
llama_model_load: mem required  = 5809.78 MB (+ 2052.00 MB per state)
llama_model_load: loading tensors from '/Users/lorenzo/Documents/repos/msc-thesis-llm-clustering/.models/gpt4all-7B/gpt4all-converted.bin'
llama_model_load: model size =  4017.27 MB / num tensors

In [11]:
gpt4all.init_prompt(template=PROMPT_TEMPLATE, input_vars=['tweet'])

2023-04-04 17:25:10,958 - INFO     | models.generation | Injecting Variables: ['tweet']


'Answer the question based on the context below.     Context: You are a marketing and customer relationship management assistant,     your task is to classify a given tweet as either a     potential lead or not. Provide your analysis of the following social media post (tweet)     as a potential lead in the context of marketing and customer relationship management.     Tweet: {tweet}     Question: Is the above tweet a potential lead? Yes or No? Why?.     Answer: '

In [12]:
logger.info('Starting GPT4All generation...')
gpt4all_outs = np.zeros(N_TWEETS, dtype=object)
for i, tweet in enumerate(tqdm(output_tweets['full_text'])):
    gpt4all_llm, gpt4all_out = gpt4all.generate(tweet)
    gpt4all_outs.append(gpt4all_out)
    time.sleep(FAST_COOLDOWN)
    if (i+1) % 50 == 0:
        # save checkpoint and cooldown
        output_tweets['gpt4all_out'] = gpt4all_outs
        save_to_parquet(data_dir='outputs', df=output_tweets, name='output_tweets')
        time.sleep(COOLDOWN)

2023-04-04 17:25:10,981 - INFO     | __main__   | Starting GPT4All generation...


  0%|          | 0/100 [00:00<?, ?it/s]

In [13]:
output_tweets['gpt4all_out'] = np.array(gpt4all_outs)
save_to_parquet(data_dir='outputs', df=output_tweets, name='output_tweets')

### Llama 7B

In [ ]:
llama_7b = Model(
    model_name='llama',
    n_threads=6,
    local_model_path=LLAMA_7B_PATH
)

In [ ]:
llama_7b.init_prompt(template=PROMPT_TEMPLATE, input_vars=['tweet'])

In [ ]:
logger.info('Starting Llama 7B generation...')
llama_7b_outs = []
output_tweets['llama_7b_out'] = ''
for i, tweet in enumerate(tqdm(output_tweets['full_text'])):
    llama_7b_llm, llama_7b_out = llama_7b.generate(tweet)
    llama_7b_outs.append(llama_7b_out)
    time.sleep(FAST_COOLDOWN)
    if i % 50:
        # save checkpoint and cooldown
        output_tweets['llama_7b_out'] = np.array(llama_7b_outs)
        save_to_parquet(data_dir='outputs', df=output_tweets, name='output_tweets')
        time.sleep(COOLDOWN)

In [ ]:
output_tweets['llama_7b_out'] = np.array(llama_7b_outs)
save_to_parquet(data_dir='outputs', df=output_tweets, name='output_tweets')

### Llama 13B

In [ ]:
llama_13b = Model(
    model_name='llama',
    n_threads=6,
    local_model_path=LLAMA_13B_PATH
)

In [ ]:
llama_13b.init_prompt(template=PROMPT_TEMPLATE, input_vars=['tweet'])

In [ ]:
logger.info('Starting Llama 13B generation...')
llama_13b_outs = []
output_tweets['llama_13b_out'] = ''
for i, tweet in enumerate(tqdm(output_tweets['full_text'])):
    llama_13b_llm, llama_13b_out = llama_13b.generate(tweet)
    llama_13b_outs.append(llama_13b_out)
    time.sleep(FAST_COOLDOWN)
    if i % 50:
        # save checkpoint and cooldown
        output_tweets['llama_13b_out'] = np.array(llama_13b_outs)
        save_to_parquet(data_dir='outputs', df=output_tweets, name='output_tweets')
        time.sleep(COOLDOWN)

In [ ]:
output_tweets['llama_13b_out'] = np.array(llama_13b_outs)
save_to_parquet(data_dir='outputs', df=output_tweets, name='output_tweets')